In [1]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.pipeline import make_pipeline
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from sklearn.preprocessing import StandardScaler

In [2]:
#path1 = "https://raw.githubusercontent.com/jaarciah/Proyecto-Integrador-2020-periodo-2/master/Data/Preprocessing/hurto_motos_categorias_ingCaracteristicas.csv"
path2 = "https://raw.githubusercontent.com/jaarciah/Proyecto-Integrador-2020-periodo-2/master/Data/Modeling/hurto_preprocesado_C.csv"

#hurtos_moto = pd.read_csv(path1, sep=";", low_memory=False)
hurto_preprocesado = pd.read_csv(path2, sep=",", low_memory=False)

In [3]:
hurto_preprocesado.modalidad.value_counts()

0    33100
1    25455
Name: modalidad, dtype: int64

In [4]:
hurto_preprocesado.head()

,latitud,longitud,estado_civil,medio_transporte,modalidad,nombre_barrio,codigo_comuna,lugar,sede_receptora,modelo,...,anho,mes,dia,dia_semana,hora,festivos,quincena,week_number,ferias_fiestas,franja_horaria
0,6.268040,-75.549892,0,0,0,0,0,0,0,0,...,2017,1,1,6,23,1,1,52,0,2
1,6.268129,-75.557882,0,0,0,4,3,2,3,4,...,2017,1,1,6,16,1,1,52,0,1
2,6.268399,-75.549576,0,0,0,0,0,0,0,4,...,2017,1,1,6,10,1,1,52,0,0
3,6.263576,-75.558601,3,0,0,4,3,0,3,4,...,2017,1,3,1,8,0,0,1,0,0
4,6.263299,-75.556534,0,1,1,4,3,0,3,0,...,2017,1,7,5,3,0,0,1,0,2


In [5]:
hurto_preprocesado.columns

Index(['latitud', 'longitud', 'estado_civil', 'medio_transporte', 'modalidad',
       'nombre_barrio', 'codigo_comuna', 'lugar', 'sede_receptora', 'modelo',
       'fecha', 'anho', 'mes', 'dia', 'dia_semana', 'hora', 'festivos',
       'quincena', 'week_number', 'ferias_fiestas', 'franja_horaria'],
      dtype='object')

In [6]:
# variables escogidas despues del proceso de ingenieria de caracteristica.
variables = ["latitud","longitud","estado_civil","medio_transporte","modalidad","nombre_barrio","codigo_comuna","lugar","sede_receptora","modelo", "fecha"]

hurto_preprocesado_new = hurto_preprocesado[variables]


# Proceso de cross-validation.

In [7]:
y = hurto_preprocesado_new.modalidad
x = hurto_preprocesado_new.drop(['modalidad'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=8)

# 1. Regresión logistica binaria.

In [45]:
modelRLB = Pipeline([('logisticRegression', LogisticRegression())
                    ])

C = np.logspace(-4, 4, 4)

params = {'LogisticRegression__penalty':("l1", "l2", "elasticnet"),
         'LogisticRegression__dual':(True, False),
          'LogisticRegression__C':(C),
          'LogisticRegression__solver':('newton-cg', 'liblinear')
         }

In [46]:
grid_modelRLB = GridSearchCV(modelRLB, params, cv=5)

In [50]:
grid_modelRLB.fit(X_train,y_train)

In [36]:
ypred = grid_modelRLB.predict(X_test)

In [37]:
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.68      0.75      0.71      6657
           1       0.62      0.53      0.57      5054

    accuracy                           0.66     11711
   macro avg       0.65      0.64      0.64     11711
weighted avg       0.65      0.66      0.65     11711



In [40]:
grid_modelRLB.best_estimator_

Pipeline(memory=None,
         steps=[('LogisticRegression',
                 LogisticRegression(C=21.54434690031882, class_weight=None,
                                    dual=False, fit_intercept=True,
                                    intercept_scaling=1, l1_ratio=None,
                                    max_iter=100, multi_class='auto',
                                    n_jobs=None, penalty='l2',
                                    random_state=None, solver='newton-cg',
                                    tol=0.0001, verbose=0, warm_start=False))],
         verbose=False)

In [38]:
# Save to file in the current working directory
pkl_filename = "/home/jaarciah/PI/Clasificación /modelos/iteración_3/grid_modelRLB.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(grid_modelRLB, file)

# 2. SVM.

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [10]:
modelSvm = Pipeline([('SVC', svm.SVC())
                    ])
C = np.logspace(-2, 10, 2)
gamma_range = np.logspace(-9, 3, 2)

params = {'SVC__C':(C),
          'SVC__kernel':('linear', 'poly', 'rbf', 'sigmoid', 'precomputed'),
          'SVC__gamma':(gamma_range)
          }

In [11]:
grid_modelSvm = GridSearchCV(modelSvm, params, cv=5)

In [ ]:
grid_modelSvm.fit(X_train,y_train)

In [ ]:
ypred = grid_modelSvm.predict(X_test)

In [ ]:
print(classification_report(y_test, ypred))

In [ ]:
grid_modelSvm.best_estimator_

In [ ]:
# Save to file in the current working directory
pkl_filename = "/home/jaarciah/PI/Clasificación /modelos/iteración_3/grid_modelSvm.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(grid_modelSvm, file)